In [ ]:
import os
import csv
import re

# Liste des mots à ignorer (stop words) en français et en anglais
stop_words = set([
    'le', 'la', 'les', 'un', 'une', 'des', 'du', 'de', 'au', 'aux', 'ce', 'cette', 'ces',
    'mon', 'ton', 'son', 'notre', 'votre', 'leur', 'et', 'ou', 'mais', 'donc', 'or', 'ni',
    'car', 'à', 'de', 'en', 'sur', 'sous', 'chez', 'dans', 'par', 'pour', 'the', 'a', 'an', 
    'and', 'or', 'but', 'so', 'because', 'of', 'in', 'on', 'at', 'with', 'by', 'for'
])

# Initialiser les variables pour les identifiants
global_subtitle_id = 1
movie_id_map = {}
subtitles = []

def clean_text(text):
    # Nettoyer le texte en supprimant les mots à ignorer
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def process_srt_file(file_path, movie_title):
    global global_subtitle_id
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Traitement des lignes de sous-titres
    subtitle_text = ''
    for line in lines:
        if line.strip() and not re.match(r'^\d+$', line.strip()):  # Ignorer les numéros de sous-titres et les lignes vides
            subtitle_text += line.strip() + ' '
    
    subtitle_text = clean_text(subtitle_text.strip())
    if subtitle_text:
        subtitles.append({
            'subtitle_id': global_subtitle_id,
            'movie_id': movie_id_map[movie_title],
            'movie_title': movie_title,
            'subtitle_text': subtitle_text
        })
        global_subtitle_id += 1

def traverse_directories(base_path):
    global movie_id_map
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.srt'):
                # Extraire le titre du film depuis le nom du fichier
                movie_title_match = re.search(r'^(.*?)s\d+', file)
                if movie_title_match:
                    movie_title = movie_title_match.group(1).strip()
                    if movie_title not in movie_id_map:
                        movie_id_map[movie_title] = len(movie_id_map) + 1
                
                # Traiter le fichier .srt
                process_srt_file(os.path.join(root, file), movie_title)

def write_csv(output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['subtitle_id', 'movie_id', 'movie_title', 'subtitle_text'])
        for subtitle in subtitles:
            writer.writerow([
                subtitle['subtitle_id'],
                subtitle['movie_id'],
                subtitle['movie_title'],
                subtitle['subtitle_text']
            ])

# Main execution
base_directory = 'sous-titres'
output_csv = 'subtitles_cleaned.csv'

traverse_directories(base_directory)
write_csv(output_csv)
